In [11]:
import numpy as np
import pandas as pd
from docplex.cp.model import *
import math
import random
from multiprocessing import Pool

In [12]:
taxi=pd.read_csv("taxi2000.csv")
pay=taxi['trip_distance'].tolist()

mu1=10
mu2=10

lam1=0.5
lam2=0.5
lam3=0

In [13]:
def cplexF1_a5e3(pay_tem,icount,jcount):
    
    male=range(0,jcount,2)
    female=range(1,jcount,2)

    mu1=10
    mu2=10

    lam1=0.5
    lam2=0.5
    lam3=0

    dis=np.random.rand(icount,jcount)

    mdl = CpoModel('F1')
    M=integer_var_list(icount*jcount, 0, 1, "M")
    u=float_var_list(jcount,-100, 100, 'u')

    intra5=sum( (1-(u[j]+0.01)/1.01)**2 for j in range(jcount))
    muf=sum( (u[j]+0.01)/1.01 for j in female)/len(female)
    mum=sum( (u[j]+0.01)/1.01 for j in male)/len(male)
    inter3=(mum-muf)**2
    CustCare=sum(M[i*jcount+j]*dis[i,j] for i in range(icount) for j in range(jcount))

    obj=lam1*intra5+lam2*inter3+lam3*CustCare

    for j in range(jcount):
        mdl.add(1-sum(M[i*jcount+j] for i in range(icount)) >= 0)
        mdl.add(sum(M[i*jcount+j]*(pay_tem[i]-dis[i,j]) for i in range(icount)) == u[j])

    for i in range(icount):
        mdl.add(sum(M[i*jcount+j] for j in range(jcount))==1)

    mdl.minimize(obj)
    msol = mdl.solve(LogVerbosity='Terse') 
    
    return msol.get_solve_time()

def cplexF1_a5e0(pay_tem,icount,jcount ):
    # kdd 
    male=range(0,jcount,2)
    female=range(1,jcount,2)

    #dis=random.random((icount,jcount))
    dis=np.random.rand(icount,jcount)

    mdl = CpoModel('F1')
    M=integer_var_list(icount*jcount, 0, 1, "M")
    u=float_var_list(jcount,-100, 100, 'u')

    intra5=sum( (1-(u[j]+0.01)/1.01)**2 for j in range(jcount))
    inter0=0
    CustCare=sum(M[i*jcount+j]*dis[i,j] for i in range(icount) for j in range(jcount))

    obj=lam1*intra5+lam2*inter0+lam3*CustCare

    for j in range(jcount):
        mdl.add(1-sum(M[i*jcount+j] for i in range(icount)) >= 0)
        mdl.add(sum(M[i*jcount+j]*(pay_tem[i]-dis[i,j]) for i in range(icount)) == u[j])

    for i in range(icount):
        mdl.add(sum(M[i*jcount+j] for j in range(jcount))==1)

    mdl.minimize(obj)
    msol = mdl.solve(LogVerbosity='Terse') #msol = mdl.solve() #

    return msol.get_solve_time()

In [18]:
repeat=5
ran=range(18,19)
inter_name=['Sühr et al. 2019','Intra 5 + Inter 1','Intra 5 + Inter 2','Intra 5 + Inter 3']
parity_inter3=[pd.DataFrame(columns=['size','runtime'])]*repeat
parity_inter0=[pd.DataFrame(columns=['size','runtime'])]*repeat
parity_con3=pd.DataFrame(columns=['size','runtime'])
parity_con0=pd.DataFrame(columns=['size','runtime'])

def mp(r):
    #for r in range(repeat):
    for t in ran:
        pay_start=random.randint(0,1999-t+10)
        runtime3=cplexF1_a5e3(pay[pay_start:pay_start+t-10],t-10,t)
        runtime0=cplexF1_a5e0(pay[pay_start:pay_start+t-10],t-10,t)
        parity_inter3[r]=parity_inter3[r].append({'size':t,'runtime':runtime3}, ignore_index=True)
        parity_inter0[r]=parity_inter0[r].append({'size':t,'runtime':runtime0}, ignore_index=True)
    return parity_inter3[r],parity_inter0[r]

with Pool(repeat) as p: 
    for result3,result0 in p.map(mp,range(repeat)):
        #print(result)
        parity_con3=pd.concat([parity_con3,result3],ignore_index=True)
        parity_con0=pd.concat([parity_con0,result0],ignore_index=True)

#parity_inter3['size']=parity_inter3['size'].astype('int32')
parity_con3.to_csv('runtime_inter3_jupyter'+str(ran[0:-1])+'.csv',index=False)
#parity_inter0['size']=parity_inter0['size'].astype('int32')
parity_con0.to_csv('runtime_kdd_jupyter'+str(ran[0:-1])+'.csv',index=False)

 ! --------------------------------------------------- CP Optimizer 20.1.0.1 --
 ! Minimization problem - 144 variables, 44 constraints
 ! Presolve      : 70 extractables eliminated
 ! LogVerbosity         = Terse
 ! --------------------------------------------------- CP Optimizer 20.1.0.1 --
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 144.0 (before), 144.0 (after)
 ! Minimization problem - 144 variables, 44 constraints
 !  . Memory usage      : 515.1 kB (before), 515.1 kB (after)
 ! --------------------------------------------------- CP Optimizer 20.1.0.1 --
 ! Presolve      : 70 extractables eliminated
 ! Using parallel search with 48 workers.
 ! Minimization problem - 144 variables, 44 constraints
 ! LogVerbosity         = Terse
 ! ----------------------------------------------------------------------------
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 ! Presolve      : 70 extractables eliminated
 !      

 *      7.027849     9199  3.30s       23      (gap is 89.74%)
 ! Time = 3.30s, Average fail depth = 58, Memory usage = 53.4 MB
 ! Current bound is 0.7207662 (gap is 89.74%)
 !          Best Branches  Non-fixed    W       Branch decision
 *      7.017091     9467  3.30s       23      (gap is 89.73%)
 *      7.016855     9760  3.30s       23      (gap is 89.73%)
 *      6.103666     9381  3.51s        8      (gap is 96.94%)
 *      6.993851    10014  3.58s       23      (gap is 89.69%)
        63.42214     9871         11   21   F     1 != M_113
 + New bound is 53.60457 (gap is 15.48%)
 *      6.017873    10126  3.74s       19      (gap is 96.90%)
 *      6.003922    10136  3.74s       19      (gap is 96.89%)
 *      5.991241    10217  3.74s       19      (gap is 96.88%)
 *      5.986808    10234  3.74s       19      (gap is 96.88%)
 *      6.988402    10380  3.86s       35      (gap is 89.69%)
 *      6.988402    10446  3.86s       35      (gap is 89.69%)
 *      63.26056    10811  3.8

 !          Best Branches  Non-fixed    W       Branch decision
                        0        144                 -
 + New bound is 0
                        0        144    1            -
 + New bound is 40.34631
 *      71.37383      132  0.46s        1      (gap is 43.47%)
 *      71.26560      165  0.46s        1      (gap is 43.39%)
 *      71.00830      225  0.46s        1      (gap is 43.18%)
 *      66.27752      242  0.46s        1      (gap is 39.13%)
 *      64.67640      439  0.46s        1      (gap is 37.62%)
 *      63.88455      452  0.46s        8      (gap is 36.84%)
 *      63.40281     1745  0.60s       11      (gap is 36.37%)
 *      63.34290     2369  0.77s       48      (gap is 36.30%)
 *      63.26855     3681  0.96s        8      (gap is 36.23%)
 *      63.16862     3124  0.96s       18      (gap is 36.13%)
 *      62.94863     3397  0.96s       18      (gap is 35.91%)
        62.94863     4713          9   17   F     0 != M_49
 + New bound is 52.36081 (gap 

 ! --------------------------------------------------- CP Optimizer 20.1.0.1 --
 ! Minimization problem - 144 variables, 44 constraints
 ! Presolve      : 70 extractables eliminated
 ! LogVerbosity         = Terse
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 144.0 (before), 144.0 (after)
 !  . Memory usage      : 514.8 kB (before), 514.8 kB (after)
 ! Using parallel search with 48 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        144                 -
 + New bound is 0
                        0        144    1   F        -
 + New bound is 0.01206532
 *      7.762334      132  0.33s        1      (gap is 99.84%)
 *      7.341062      145  0.33s        1      (gap is 99.84%)
 *      7.272969      261  0.33s        1      (gap is 99.83%)
 *      6.842154      934  0.33s        2      (gap is 99.82%)
 *  

In [8]:
result3

,size,runtime
0,18.0,472.7


In [9]:
result0

,size,runtime
0,18.0,157.36


In [22]:
parity_con3

,size,runtime
0,18.0,3844.61
1,18.0,925.61
2,18.0,343.53
3,18.0,3071.50
4,18.0,2442.56


In [23]:
parity_con0

,size,runtime
0,18.0,598.40
1,18.0,148.06
2,18.0,170.83
3,18.0,648.85
4,18.0,405.23
